## This file has now been converted to a python class (dataPrep) 
<br>
Below is a sample of what the class does. 

In [1]:
import sys
sys.path.insert(0, "/Users/shah/CODE_BOOK_3/code_document/MyRepo/THESIS")
sys.modules.pop("src", None)

In [2]:
import pandas as pd
import numpy as np
from math import sqrt, pi, exp
from arch import arch_model
import yfinance as yf 
from scipy.optimize import minimize
import warnings
from arch.__future__ import reindexing
from arch.utility.exceptions import ConvergenceWarning
from src.msGarch import msGARCH
from src.msGarch import msGARCHIV
from tqdm import tqdm
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", message="y is poorly scaled")
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)


## Data

In [3]:
d_btc = pd.read_parquet("../../data/rawData/btc_underlying_minutely_2025.parquet")
df = pd.read_parquet("../../data/rawData/derivative_data_2025.parquet")


In [4]:
d_btc = d_btc.set_index("date_time", drop = False)

In [5]:
rfr = yf.download("^IRX", start=d_btc.index.min().date() - pd.Timedelta("1D"), end=d_btc.index.max().date(), progress=False)["Close"].dropna() / 100
d_btc["rfr"] = rfr.tz_localize("UTC").reindex(d_btc.index.normalize(), method="ffill").to_numpy()


In [6]:
d_btc = d_btc.resample("1h").last()


In [7]:
d_btc['r'] = np.log(d_btc["close"]).diff()

In [8]:
df["strike"] = df["instrument_name"].str.split("-").str[2].astype(float)
df["type"] = df["instrument_name"].str.split("-").str[3].astype(str)
df["option_price"] = df.price * df.index_price


In [9]:
# get maturity from instrument_name
df["maturity"] = pd.to_datetime(df.instrument_name.str.split("-").str[1], format = "%d%b%y")

In [10]:
df = df.set_index("date_time", drop = False)

In [11]:

rfr = yf.download("^IRX", start = df.index.min().normalize() - pd.Timedelta("1D"), end= df.index.max().normalize()) ["Close"]/ 100
df["rfr"] = rfr.reindex(df.index, method = "ffill")


[*********************100%***********************]  1 of 1 completed


In [12]:
df.columns

Index(['trade_seq', 'trade_id', 'timestamp', 'tick_direction', 'price',
       'mark_price', 'iv', 'instrument_name', 'index_price', 'direction',
       'contracts', 'amount', 'combo_trade_id', 'combo_id',
       'block_trade_leg_count', 'block_trade_id', 'block_rfq_id',
       'liquidation', 'date_time', 'strike', 'type', 'option_price',
       'maturity', 'rfr'],
      dtype='str')

In [13]:
d_btc = d_btc.rename(columns={"close": "Close"})

In [14]:
# Save two READY data frames. 
file_path0 = "../../data/rawData/d_2025.parquet" # contains stock price. Uniform Index
file_path1 = "../../data/rawData/df_2025.parquet" # # contains order book. Non-Uniform Index
d_btc.to_parquet(file_path0)
df.to_parquet(file_path1)